In [ ]:
# ??
import backtrader as bt
from datetime import datetime
from collections import OrderedDict
import os
import time

class firstStrategy(bt.Strategy):

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)

    def next(self):
        if not self.position:
            if self.rsi < 30:
                self.buy(size=100)
        else:
            if self.rsi > 70:
                self.sell(size=100)


def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    '''
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

def printSQN(analyzer):
    sqn = round(analyzer.sqn,2)
    print('SQN: {}'.format(sqn))

#Variable for our starting cash
startcash = 100000

#Create an instance of cerebro
cerebro = bt.Cerebro()

#Add our strategy
cerebro.addstrategy(firstStrategy)

#Get Apple data from Yahoo Finance.
os.chdir('C:\\Users\\asus\\Documents\\Python Projects\\Trading - Research and Backtest\\AAPL')
datapath = os.getcwd() + '\\Databases\\'
data     = bt.feeds.GenericCSVData(dataname = datapath + 'AAPL-5min.csv',
                                   timeframe = bt.TimeFrame.Minutes,
                                   compression = 5,
                                   #sessionstart = datetime.time(9, 35, 0),
                                   #sessionend = datetime.time(15, 30, 0),
                                   reverse = False, datetime = 0, time = 6,
                                   high = 2, low = 3, open = 1, close = 4, volume = 5, openinterest = -1,
                                   nullvalue = 0.0, #fromdate = datetime.datetime(2020, 4, 15), todate = datetime.datetime(2020, 4, 16),
                                   dtformat = ('%Y-%m-%d'), tmformat = ('%H:%M:%S')
                                  )

#Add the data to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(startcash)

# Add the analyzers we are interested in
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")

# Run over everything
strategies = cerebro.run()
firstStrat = strategies[0]

# print the analyzers
printTradeAnalysis(firstStrat.analyzers.ta.get_analysis())
printSQN(firstStrat.analyzers.sqn.get_analysis())

#Get final portfolio Value
portvalue = cerebro.broker.getvalue()

#Print out the final result
print('Final Portfolio Value: ${}'.format(portvalue))

#Finally plot the end results
cerebro.plot(style='candlestick')

# Libraries

In [1]:
# Libraries
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import seaborn as sns
import datetime
from pandas.tseries.offsets import DateOffset
import time
import decimal
import os
import scipy.stats as stats
from scipy.stats import norm
import plotly.graph_objs as go
import plotly.offline as pyoff
from jupyterthemes import jtplot
jtplot.style(theme='onedork')

# Correlations and Covariances

In [2]:
# Move columns functions
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    return(df[seg1 + seg2 + seg3])

In [3]:
# List of US Stocks
us_tickers = ['AAPL', 'AMZN', 'FB', 'NFLX', 'SPY', 'TSLA', 'V']
equities = {}
for ticker in us_tickers:
    equities.update({ticker: {}})
    path = 'C:\\Users\\asus\\Documents\\Python Projects\\Trading - Research and Backtest\\'+ticker
    os.chdir(path)
    datapath = os.getcwd() + '\\Databases\\'
    files = []
    for (dirpath, dirnames, filenames) in os.walk(datapath):
        files.extend(filenames)
        break
    for file in files:
            _ = pd.read_csv(datapath + file, index_col = 'date', parse_dates = True)
            equities[ticker].update({file.split('-')[1].split('.')[0]: _})

ValueError: 'date' is not in list

In [ ]:
# Merge Data
start = '2020-06-01'
end = '2020-07-28'
intervals = ['5min', '15min', '30min', '60min', 'daily']
merge_df = {}
for interval in intervals:
    if interval == 'daily':
        _merge = equities['AAPL'][interval][['4. close']]\
                     .loc[start:end].rename(columns={'4. close':'4. close_AAPL'}).reset_index()
        for ticker in us_tickers:
            if not ticker == 'AAPL':
                b = equities[ticker][interval][['4. close']]\
                    .loc[start:end].rename(columns={'4. close':('4. close_'+ticker)}).reset_index()
                _merge = _merge.merge(b, left_on=['date'], right_on=['date'], how='outer')
        _merge.set_index('date', inplace=True)
        merge_df.update({interval: _merge})
    else:
        _merge = equities['AAPL'][interval][['4. close', 'time']]\
                     .loc[start:end].rename(columns={'4. close':'4. close_AAPL'}).reset_index()
        for ticker in us_tickers:
            if not ticker == 'AAPL':
                b = equities[ticker][interval][['4. close', 'time']]\
                    .loc[start:end].rename(columns={'4. close':('4. close_'+ticker)}).reset_index()
                _merge = _merge.merge(b, left_on=['date', 'time'], right_on=['date', 'time'], how='outer')
        _merge['datetime'] = pd.to_datetime(_merge['date'].astype(str)+' '+_merge['time'])
        _merge = movecol(_merge, cols_to_move=['datetime', 'date', 'time'], ref_col='4. close_AAPL', place='Before')\
                 .set_index('datetime')
        merge_df.update({interval: _merge})

### Covariance

In [ ]:
# Find Covariance between each stocks closing price
covariance = merge_df['daily'].dropna().cov()
# Overall Plot
plot = sns.heatmap(covariance,cmap='coolwarm',annot=True)
plot.set(ylim=(len(covariance)-0.005, -0.05))

In [ ]:
# Individual Plot
vec1 = covariance['4. close_V']
fig, ax = plt.subplots()
plot = sns.heatmap([vec1], cmap='coolwarm', annot=True, xticklabels=vec1.index.to_list())

### Correlations

In [ ]:
# Find Correlation between each stocks closing price
interval = 'daily'
correlation = merge_df[interval].dropna().corr(method='pearson')
print('Correlation of {} closing price of stocks for the period {} to {}'.format(interval, start, end))
# Overall Plot
plt.subplots(figsize=(10, 6))
plot = sns.heatmap(correlation,cmap='coolwarm',annot=True)
plot.set(ylim=(len(correlation)-0.005, -0.05))
plt.show()

In [ ]:
# Individual Plot
base_stock = 'AAPL'
vec1 = correlation['4. close_'+base_stock]
print('Correlation of {} closing price for the period {} to {} towards other equities'.format(base_stock, start, end))
fig, ax = plt.subplots(figsize=(10, 6))
plot = sns.heatmap([vec1], cmap='coolwarm', annot=True, xticklabels=vec1.index.to_list())

In [ ]:
# Rolling Individual Plot
plot_data = []
window = 5
for ticker in us_tickers:
    rolling_r = merge_df[interval]['4. close_'+base_stock].rolling(window=window, center=False)\
                .corr(merge_df[interval]['4. close_'+ticker])
    _ = go.Scatter(x=rolling_r.dropna().index.to_list(), y=rolling_r.dropna(), name=base_stock+' and '+ticker)
    plot_data.append(_)
plot_layout = go.Layout(title = 'Rolling correlation of %s stock ' % base_stock)
fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)

# Granger Causality

## Time Lagged Cross Correlation — assessing signal dynamics

In [ ]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

In [ ]:
# Leader Follower Dynamic
d1 = merge_df[interval]['4. close_'+'AAPL']
d2 = merge_df[interval]['4. close_'+'SPY']
rs = [crosscorr(d1,d2, lag) for lag in range(-int(len(d1)/2),int(len(d1)/2)+1)]
offset = np.ceil(len(rs)/2)-np.argmax(rs)
f,ax=plt.subplots(figsize=(14,3))
ax.plot(rs)
ax.axvline(np.ceil(len(rs)/2)-1,color='k',linestyle='--',label='Center')
ax.axvline(np.argmax(rs),color='r',linestyle='--',label='Peak synchrony')
ax.set(title=f'Offset = {int(offset)} \n{base_stock} leads <> SPY leads', xlabel='Offset',ylabel='Pearson r')
ax.set_xticklabels([int(item-int(len(d1)/2)) for item in ax.get_xticks()]);
plt.legend()

In [ ]:
# Rolling window time lagged cross correlation
window_size =  14 #samples
t_start = 0
t_end = t_start + window_size
step_size = 1
rss=[]
while t_end < len(merge_df[interval]):
    d1 = merge_df[interval]['4. close_'+'AAPL'].iloc[t_start:t_end]
    d2 = merge_df[interval]['4. close_'+'SPY'].iloc[t_start:t_end]
    rs = [crosscorr(d1,d2, lag, wrap=False) for lag in range(-int(window_size/2),int(window_size/2)+1)]
    rss.append(rs)
    t_start = t_start + step_size
    t_end = t_end + step_size
rss = pd.DataFrame(rss)

f,ax = plt.subplots(figsize=(10,10))
sns.heatmap(rss,cmap='RdBu_r',ax=ax, annot=True)
ax.set(title=f'Rolling Windowed Time Lagged Cross Correlation',xlabel='(<-- '+base_stock+')  Offset  (SPY -->)',ylabel='Epochs', 
       ylim=(len(rss)-0.005, -0.05))
ax.set_xticklabels(np.arange(-int(window_size/2), int(window_size/2)+1));

In [ ]:
# Time Lagged summary
_1 = rss.iloc[:, :7].mean(axis=1)
_2 = rss.iloc[:, 8:-1].mean(axis=1)
_3 = rss.iloc[:, 7]
summary = pd.DataFrame([_1, _3, _2], index=['Lead AAPL', 'Neutral', 'Lead SPY']).T
f,ax = plt.subplots(figsize=(10,10))
sns.heatmap(summary,cmap='coolwarm',ax=ax, annot=True)
ax.set(ylim=(len(rss)-0.005, -0.05))